## Scenario Visualization

In [1]:
import os
import sys
import pandas as pd
import network_wrangler as wr


%config IPCompleter.greedy=True
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

### Define Base Files, Networks, and Scenario

In [2]:
STPAUL_DIR = os.path.join(os.getcwd(),'../','examples','stpaul')
STPAUL_SHAPE_FILE = os.path.join(STPAUL_DIR,"shape.geojson")
STPAUL_LINK_FILE = os.path.join(STPAUL_DIR,"link.json")
STPAUL_NODE_FILE = os.path.join(STPAUL_DIR,"node.geojson")

In [3]:
road_net = wr.RoadwayNetwork.read(
    link_file= STPAUL_LINK_FILE, 
    node_file=STPAUL_NODE_FILE, 
    shape_file=STPAUL_SHAPE_FILE, 
    fast=True
)

2020-01-24 21:04:25, INFO: Reading from following files:
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/node.geojson
-/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/shape.geojson.
2020-01-24 21:04:31, INFO: Read 1921801 links from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/link.json
2020-01-24 21:04:31, INFO: Read 205908 nodes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/node.geojson
2020-01-24 21:04:31, INFO: Read 331345 shapes from /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/shape.geojson


In [4]:
transit_net = wr.TransitNetwork.read(STPAUL_DIR)

2020-01-24 21:04:45, INFO: Removing calendar.txt from transit network config because file not found
2020-01-24 21:04:45, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-01-24 21:04:45, INFO: Removing fare_attributes.txt from transit network config because file not found
2020-01-24 21:04:45, INFO: Removing fare_rules.txt from transit network config because file not found
2020-01-24 21:04:46, INFO: Removing feed_info.txt from transit network config because file not found


In [5]:
base_scenario = {
    "road_net": road_net,
    "transit_net": transit_net,
}

### Build a Scenario by Specifying Specific Project Cards

In [6]:
BUILD_CARD_FILENAMES = [
    '3_multiple_roadway_attribute_change.yml',
    'multiple_changes.yml',
    '4_simple_managed_lane.yml',
]

project_cards_list = [
    wr.ProjectCard.read(os.path.join(STPAUL_DIR, "project_cards", filename), validate=False)
    for filename in BUILD_CARD_FILENAMES
]

In [9]:
my_scenario = wr.Scenario.create_scenario(
    base_scenario=base_scenario, 
    project_cards_list=project_cards_list
)
my_scenario.apply_all_projects()

2020-01-24 21:09:22, INFO: Creating Scenario
2020-01-24 21:09:22, DEBUG: Adding project cards from List.
6th Street Transitway,road diet on one way couplet 5th and 6th,test managed lane project
2020-01-24 21:09:22, DEBUG: Ordered Project Cards: {'test managed lane project': <network_wrangler.ProjectCard.ProjectCard object at 0x1221a19e8>, 'road diet on one way couplet 5th and 6th': <network_wrangler.ProjectCard.ProjectCard object at 0x1221c0dd8>, '6th street transitway': <network_wrangler.ProjectCard.ProjectCard object at 0x1224822e8>}
2020-01-24 21:09:22, DEBUG: Project Cards: [<network_wrangler.ProjectCard.ProjectCard object at 0x1221a19e8>, <network_wrangler.ProjectCard.ProjectCard object at 0x1221c0dd8>, <network_wrangler.ProjectCard.ProjectCard object at 0x1224822e8>]
2020-01-24 21:09:22, DEBUG: else {'project': 'test managed lane project', 'category': 'Parallel Managed lanes', 'tags': ['example'], 'dependencies': {'prerequisites': [], 'corequisites': [], 'conflicts': []}, 'facili

2020-01-24 21:09:23, DEBUG: finished ox.gdfs_to_graph()
2020-01-24 21:09:23, DEBUG: Calculating NX shortest path from A_id: 62146 to B_id: 45691
2020-01-24 21:09:23, DEBUG: Shortest path successfully routed
2020-01-24 21:09:23, WARNING: Existing value defined for lanes in project card does not match the value in the roadway network for the selected links


In [10]:
my_scenario.applied_projects

[]

In [11]:
for p in project_cards_list:
    project_card_dictionary = p.__dict__
    summary = my_scenario.applied_project_card_summary(project_card_dictionary)
    print("-----------------------------------------------------------------------")
    print("Project Card:", summary.get("project card"))
    
    for i in range(0,summary["total_parts"]):
        pc_summary = summary["Part " + str(i+1)]
        print("Project:", pc_summary.get("project"))
        print("Category:", pc_summary.get("category"))
        print("")
    
        category = pc_summary.get("category")
    
        if category == "roadway property change" or category == "parallel managed lanes":
            print("Build Scenario:")
            sce_links_df = my_scenario.road_net.links_df.loc[
                pc_summary.get("sel_indices"), pc_summary.get("attributes")
            ]
            print(sce_links_df)
            print("")

            print("Base Scenario:")
            base_links_df = my_scenario.base_scenario["road_net"].links_df.loc[
                pc_summary.get("sel_indices"), pc_summary.get("attributes")
            ]
            print(base_links_df)
            print("")
            
            display(pc_summary.get("map"))

        elif category == "add new roadway":
            if pc_summary.get("added_links") is not None:
                print("Added Links")
                print(pc_summary.get("added_links"))
            if pc_summary.get("added_nodes") is not None:
                print("Added Nodes")
                print(pc_summary.get("added_nodes"))
            
            display(pc_summary.get("map"))
            
        elif category == "roadway deletion":
            display(pc_summary.get("map"))
            
        else:
            pass

-----------------------------------------------------------------------
Project Card: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/project_cards/3_multiple_roadway_attribute_change.yml
Project: 6th Street Transitway
Category: roadway property change

Build Scenario:
     lanes  transit_access  drive_access
1125     2               1             0
4498     2               1             0
5570     2               1             0

Base Scenario:
      lanes  transit_access  drive_access
1125      2               0             1
4498      2               0             1
5570      2               0             1



-----------------------------------------------------------------------
Project Card: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/project_cards/multiple_changes.yml
Project: road diet on one way couplet 5th and 6th – Part 1
Category: roadway property change

Build Scenario:
     lanes
1125     2
4498     2
5570     2

Base Scenario:
      lanes
1125      2
4498      2
5570      2



Project: road diet on one way couplet 5th and 6th – Part 2
Category: roadway property change

Build Scenario:
     lanes
619      1
4497     1
5235     1
6459     1

Base Scenario:
      lanes
619       3
4497      2
5235      1
6459      3



-----------------------------------------------------------------------
Project Card: /Users/elizabeth/Documents/urbanlabs/MetCouncil/working/network_wrangler/notebook/../examples/stpaul/project_cards/4_simple_managed_lane.yml
Project: test managed lane project
Category: parallel managed lanes

Build Scenario:
                                                  lanes  \
7539  {'default': 3, 'timeofday': [{'time': (21600, ...   
8104  {'default': 3, 'timeofday': [{'time': (21600, ...   

                                               ML_lanes  \
7539  {'default': 0, 'timeofday': [{'time': (21600, ...   
8104  {'default': 0, 'timeofday': [{'time': (21600, ...   

                                               ML_price ML_access ML_egress  
7539  {'default': 0, 'timeofday': [{'time': (21600, ...       all       all  
8104  {'default': 0, 'timeofday': [{'time': (21600, ...       all       all  

Base Scenario:
      lanes  ML_lanes  ML_price  ML_access  ML_egress
7539      3       NaN       